# Run locally or in QuantConnect in a .ipynb file

In [ ]:
%matplotlib inline
# Imports
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Indicators import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
# Create an instance
qb = QuantBook()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import statsmodels.api as sm
from math import floor
plt.style.use('seaborn-whitegrid')
from sklearn import linear_model

### Step 1:  Find two likely cointegrated stocks
Two stocks we choose here is XOM and CVX. They are two American multinational oil and gas corporations. They are of the same industry.

In [ ]:
syls = ["XOM","CVX"]
qb.AddEquity(syls[0])
qb.AddEquity(syls[1])
start = datetime(2003,1,1)
end = datetime(2009,1,1)
x = qb.History([syls[0]],start ,end, Resolution.Daily).loc[syls[0]]['close']
y = qb.History([syls[1]],start ,end, Resolution.Daily).loc[syls[1]]['close']
Calculate the logarithm of each price to reduce variance and adjust to a comparable scale
price = pd.concat([x, y], axis=1)
price.columns = syls 
lp = np.log(price)
Plot
price.plot(figsize = (15,10))

### Step 2: Estimate Spreads
If we have two stocks, X & Y, that are cointegrated in their price movements, then any divergence in the spread from 0 should be temporary and mean-reverting. Next step we will estimate the spread series.

In [ ]:
def reg(x, y):
    regr = LinearRegression()
    x = x.values.reshape(-1, 1)  # Convertir x en una matriz de una columna
    regr.fit(x, y)    
    beta = regr.coef_[0]  # Obtener el coeficiente beta
    alpha = regr.intercept_  # Obtener el intercepto (alpha)
    spread = y - (x.flatten() * beta + alpha)  # Calcular el spread
    return spread

# Suponiendo que 'lp' es un DataFrame con los precios
x = lp['XOM']
y = lp['CVX']

# Calcular el spread
spread = reg(x, y)

# Graficar la serie del spread
spread.plot(figsize=(15,10))
plt.ylabel('spread')
plt.show()

### Step 3: Check Stationarity
From the above plot, the first order difference $Spread_t=log(y_t) -\beta log(x_t)-\alpha$ seems to be stationary and mean-reverting. Next we will check if it is stationary. We use the ADF test to check the stationarity of the spread series.

In [ ]:
# check if the spread is stationary 
adf = sm.tsa.stattools.adfuller(spread, maxlag=1)
print('ADF test statistic: %.02f' % adf[0])
for key, value in adf[4].items():
    print('\t%s: %.3f' % (key, value))
print('p-value: %.03f' % adf[1])


### Step 4: Create Trading Signal
We have statistical significance that both assets are cointegrated. Its time to create trading signals, backtest and analyaze the results. 

# Run on QuantConnect

In [ ]:
# region imports
from AlgorithmImports import *
# endregion
from sklearn import linear_model
import numpy as np
import pandas as pd
from scipy import stats
from math import floor
from datetime import timedelta


class PairsTradingAlgorithm(QCAlgorithm):
     # Initialize the parameters
    def Initialize(self):
        
        self.SetStartDate(2009,1,1)
        self.SetEndDate(2017,1,1)
        self.SetCash(10000)
        self.numdays = 250  # set the length of training period
        tickers = ["XOM", "CVX"]
        self.symbols = []
        
        # Get Symbol and RollingWindow Objects
        self.threshold = 1.
        for i in tickers:
            self.symbols.append(self.AddSecurity(SecurityType.Equity, i, Resolution.Daily).Symbol)
        for i in self.symbols:
            i.hist_window = RollingWindow[TradeBar](self.numdays) 

    # Initialize the backtest
    def OnData(self, data):

        # Verify for symbols data availability and Add data to the RollingWindow Objects
        if not (data.ContainsKey("CVX") and data.ContainsKey("XOM")): return
        for symbol in self.symbols:
            symbol.hist_window.Add(data[symbol])
        
        # Every new rolling_window object transform to a series object
        price_x = pd.Series([float(i.Close) for i in self.symbols[0].hist_window], 
                             index = [i.Time for i in self.symbols[0].hist_window])
                             
        price_y = pd.Series([float(i.Close) for i in self.symbols[1].hist_window], 
                             index = [i.Time for i in self.symbols[1].hist_window])
        if len(price_x) < 250: return

        # Calculate the spread using a linear regression
        spread = self.regr(np.log(price_x), np.log(price_y))
        mean = np.mean(spread)
        std = np.std(spread)
        ratio = floor(self.Portfolio[self.symbols[1]].Price / self.Portfolio[self.symbols[0]].Price)
        
        
        # Define entry and exit signals
        if spread[-1] > mean + self.threshold * std:
            if not self.Portfolio[self.symbols[0]].Quantity > 0 and not self.Portfolio[self.symbols[0]].Quantity < 0:
                self.Sell(self.symbols[1], 100) 
                self.Buy(self.symbols[0],  ratio * 100) # Balance the position size with the ratio
        
        elif spread[-1] < mean - self.threshold * std:
            if not self.Portfolio[self.symbols[0]].Quantity < 0 and not self.Portfolio[self.symbols[0]].Quantity > 0:
                self.Sell(self.symbols[0], 100)
                self.Buy(self.symbols[1], ratio * 100) 

        else:
            self.Liquidate()

    
    def regr(self,x,y):
        regr = linear_model.LinearRegression()
        x_constant = np.column_stack([np.ones(len(x)), x])
        regr.fit(x_constant, y)
        beta = regr.coef_[0]
        alpha = regr.intercept_
        spread = y - x*beta - alpha
        return spread